In [1]:
import os
import warnings
import pickle as pkl
import pathlib

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker

from utils import *
from plot_utils import *


In [ ]:
def extract_results(self, save = True, normalize = True,):
    """
    Analyse the defects for all the input folders
    """
    for N in range(self.Ndata):

        defect_arr = np.nan * np.zeros((self.Nframes[N], self.Nactivity[N], self.Nexp[N]))
        av_counts = np.nan * np.zeros([self.Nframes[N], len(self.window_sizes[N]), self.Nactivity[N], self.Nexp[N]])
        var_counts = np.nan * np.zeros_like(av_counts)

        if os.path.isfile(os.path.join(self.output_paths[N], 'kbins.txt')):
            ext_sfac = True
            kbins = np.loadtxt(os.path.join(self.output_paths[N], 'kbins.txt'))
            rad = np.load(os.path.join(self.output_paths[N], 'rad.npy'))
            sfac = np.nan * np.zeros((self.Nframes[N], len(kbins), 2, self.Nactivity[N], min(10, self.Nexp[N])))
            pcf = np.nan * np.zeros((self.Nframes[N], len(rad), self.Nactivity[N], min(self.Nexp[N], 10)))
        else:
            ext_sfac = False
        
        print('Analyse defects for input folder {}'.format(self.input_paths[N]))
        for i, (act, act_dir) in enumerate(zip(self.act_list[N], self.act_dir_list[N])):
            sfac_counter = 0
            exp_list = []
            exp_dir_list = []

            for file in os.listdir(act_dir):
                exp_count = file.split('_')[-1]
                exp_list.append(int(exp_count))
                exp_dir_list.append(os.path.join(act_dir, file))

            # sort the activity list and the activity directory list
            exp_list, exp_dir_list = zip(*sorted(zip(exp_list, exp_dir_list)))

            for j, (exp, exp_dir) in enumerate(zip(exp_list, exp_dir_list)):
                defect_arr[:, i, j] = np.loadtxt(os.path.join(exp_dir, 'Ndefects_act{}_exp{}.txt'.format(act, exp)))[-self.Nframes[N]:]

                counts = np.loadtxt(os.path.join(exp_dir, 'av_counts_act{}_exp{}.txt'.format(act,exp)))[-self.Nframes[N]:,:]
                vars = np.loadtxt(os.path.join(exp_dir, 'count_fluctuations_act{}_exp{}.txt'.format(act,exp)))[-self.Nframes[N]:,:]

                idx_start_count = min(self.Nframes[N], len(counts))
                av_counts[-idx_start_count:, :, i, j] = counts[-idx_start_count:, :]
                var_counts[-idx_start_count:, :, i, j] = vars[-idx_start_count:, :]

                if ext_sfac and sfac_counter < 10:
                    try:
                        sfac[-self.Nframes[N]:, :, :, i, sfac_counter] = np.load(os.path.join(exp_dir, 'structure_factor_act{}_exp{}.npy'.format(act,exp)))[-self.Nframes[N]:, :,:]
                    except:
                        pass
                    try:
                        pcf[-idx_start_count:, :, i, sfac_counter] = np.loadtxt(os.path.join(exp_dir, 'pcf.txt'.format(act,exp)))[-self.Nframes[N]:,:]
                    except:
                        pass
                    if os.path.isfile(os.path.join(exp_dir,'sfac_analysis_completed.txt')):
                        sfac_counter += 1
    
        dens_fluc = var_counts / (np.pi * self.window_sizes[N][:, None, None] ** 2) ** 2
        if normalize:
            av_dens = defect_arr / self.LX[N] ** 2
            dens_fluc = dens_fluc / av_dens[:, None, :, :] ** 2

        if save:
            np.save(os.path.join(self.output_paths[N], 'defect_arr.npy'), defect_arr)
            np.save(os.path.join(self.output_paths[N], 'av_counts.npy'), av_counts)
            np.save(os.path.join(self.output_paths[N], 'var_counts.npy'), var_counts)
            np.save(os.path.join(self.output_paths[N], 'dens_fluc.npy'), dens_fluc)

            if ext_sfac:
                np.save(os.path.join(self.output_paths[N], 'sfac.npy'), sfac)
                np.save(os.path.join(self.output_paths[N], 'pcf.npy'), pcf)

def plot_defects_per_activity(obj, activity, ax = None, Ndataset = 0, stationarity_dict = {}, alpha = 0.55):
        
    output_path = obj.output_paths[Ndataset]

    act_idx = list(obj.act_list[Ndataset]).index(activity)   
    Nframes = obj.Nframes[Ndataset]

    try:
        defect_arr_av = obj.get_arrays_av(Ndataset = Ndataset)[0] 
    except:
        print('Defect array not found. Analyse defects first.')
        return

    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 6))
        return_fig = True


    ax.errorbar(np.arange(0, Nframes * obj.Ninfo[Ndataset], obj.Ninfo[Ndataset]), \
                        defect_arr_av[:, act_idx, 0], defect_arr_av[:, act_idx, 1], fmt='.', \
                        alpha = alpha, markersize=9, label='Activity = {}'.format(activity), color = f'C{1}') 

    if stationarity_dict != {}:
            x = est_stationarity(defect_arr_av[:, act_idx, 0], **stationarity_dict)[0]
    else:
        x = obj.conv_list[Ndataset][act_idx]
        
    if x > 0:
        ax.axvline(x, color='black', linestyle='--', alpha=0.5)

    ax.grid()
    ax.set(ylim = (0, np.max(defect_arr_av[:, act_idx, 0]) * 1.5))

    if return_fig:
        return fig, ax
    return ax

In [3]:
extract = False

data_path = 'X:\\na_ndg'
data_subdirs = [os.path.join(data_path, subdir) for subdir in os.listdir(data_path)[:-1]]
save_path = 'data\\na_ndg'
if not os.path.isdir(save_path):
    os.mkdir(save_path)

dir0 = data_subdirs[0]
window_sizes = np.loadtxt(os.path.join(dir0, 'window_sizes.txt'))
kbins = np.loadtxt(os.path.join(dir0, 'kbins.txt'))
rad = np.loadtxt(os.path.join(dir0, 'rad.txt'))

Nsim = len(os.listdir(data_path))
Nframes = 1501
Nwindows = len(window_sizes)

defect_arr = np.nan * np.zeros((Nframes, Nsim))
av_counts = np.nan * np.zeros([Nframes, Nwindows, Nsim])
var_counts = np.nan * np.zeros_like(av_counts)
sfac = np.nan * np.zeros((Nframes, len(kbins), 2, Nsim))
pcf = np.nan * np.zeros((Nframes, len(rad), Nsim))

if extract:
    for i, dirr in enumerate(data_subdirs):
        defect_arr[:, i] = np.loadtxt(os.path.join(dirr,f'Ndefects_N{i}.txt'))[-Nframes:]
        av_counts[:,:,i] = np.loadtxt(os.path.join(dirr,f'av_counts_act_N{i}.txt'))[-Nframes:]
        var_counts[:,:,i] = np.loadtxt(os.path.join(dirr,f'count_fluctuations_N{i}.txt'))[-Nframes:]
        sfac[:, :, :, i] = np.load(os.path.join(dirr,f'structure_factor_N{i}.npy'))[-Nframes:]
        pcf[:, :, i] = np.load(os.path.join(dirr,f'pcf.npy'))[-Nframes:]

    np.save(os.path.join(save_path, 'window_sizes.npy'), window_sizes)
    np.save(os.path.join(save_path, 'rad.npy'), window_sizes)
    np.save(os.path.join(save_path, 'kbins.npy'), window_sizes)

    np.save(os.path.join(save_path, 'defect_arr.npy'), defect_arr)
    np.save(os.path.join(save_path, 'av_counts.npy'), av_counts)
    np.save(os.path.join(save_path, 'var_counts.npy'), var_counts)
    np.save(os.path.join(save_path, 'sfac.npy'), sfac)
    np.save(os.path.join(save_path, 'pcf.npy'), pcf)
else: 
    window_sizes = np.load(os.path.join(save_path, 'window_sizes.npy'))
    rad = np.load(os.path.join(save_path, 'rad.npy'))
    kbins = np.load(os.path.join(save_path, 'kbins.npy'))

    defect_arr = np.load(os.path.join(save_path, 'defect_arr.npy'))
    av_counts = np.load(os.path.join(save_path, 'av_counts.npy'))
    var_counts = np.load(os.path.join(save_path, 'var_counts.npy'))
    sfac = np.load(os.path.join(save_path, 'sfac.npy'))
    pcf = np.load(os.path.join(save_path, 'pcf.npy'))


In [9]:
os.path.isfile(dict_path)

True

In [11]:
os.getcwd()

'c:\\Users\\Simon Andersen\\Projects\\Projects\\Thesis\\NematicAnalysis'

In [12]:
dict_path = os.path.join('C:\\Users\\Simon Andersen\\Projects\\Projects\\Thesis\\NematicAnalysis', save_path, 'model_params0.pkl')

In [13]:
dict_path

'C:\\Users\\Simon Andersen\\Projects\\Projects\\Thesis\\NematicAnalysis\\data\\na_ndg\\model_params0.pkl'

In [14]:
#dict_path = os.path.join(save_path, 'model_params0.pkl')
with open(dict_path, 'rb') as f:
    params = pkl.load(f)

NotImplementedError: cannot instantiate 'PosixPath' on your system

In [7]:
for i in range(1):
    
    dict_path = os.path.join(save_path, f'model_params{i}.pkl')
    with open(dict_path, 'rb') as f:
        params = pkl.load(f)
    print(params)

NotImplementedError: cannot instantiate 'PosixPath' on your system

In [ ]:
data_path = 'X:/na_ndg'
data_subdirs = [os.path.join(data_path, subdir) for subdir in os.listdir(data_path)[:-1]]

import shutil

if not os.path.isdir(save_path):
    os.mkdir(save_path)
for i, dirr in enumerate(data_subdirs):
    params_path = os.path.join(dirr,f'model_params.pkl')
    dict_path = os.path.join(save_path, f'model_params{i}.pkl')
    shutil.copyfile(params_path, dict_path)
    with open(dict_path, 'rb') as f:
        params = pkl.load(f)
    print(params)